<a href="https://colab.research.google.com/github/jahelsantiago/tensorflow_tutorial/blob/master/IMBD_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [5]:
imbd, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVU8HFL/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVU8HFL/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVU8HFL/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
imbd

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}

In [18]:
train_data, test_data = imbd["train"], imbd["test"]

train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

for s,l in train_data:
  train_sentences.append(s.numpy().decode("utf8"))
  train_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(s.numpy().decode("utf8"))
  test_labels.append(l.numpy())

train_labels = np.array(train_labels) 
test_labels = np.array(test_labels)

In [32]:
vocab_size = 10000
oov_tok = "<OOV>"
maxlen = 120

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_sequences = pad_sequences(sequences, maxlen, truncating="post")

test_sequence = tokenizer.texts_to_sequences(test_sentences)
test_sequence = pad_sequences(test_sequence, maxlen, truncating="post")

In [33]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return " ".join([reverse_word_index[i] for i in text])

print(decode_review(test_sequence[0]))
print(test_sequence[0])

there are films that make careers for george romero it was night of the living dead for kevin smith <OOV> for robert rodriguez el <OOV> add to that list <OOV> <OOV> absolutely amazing <OOV> a <OOV> less flawless film making and as assured and as professional as any of the aforementioned movies i haven't laughed this hard since i saw the full monty and even then i don't think i laughed quite this hard so to speak <OOV> talent is considerable <OOV> a <OOV> less is so chock full of double <OOV> that one would have to sit down with a copy of this script and do a line by line examination of it to fully appreciate the uh <OOV>
[  48   24  106   13   95 4066   16  740 5065   10   14  312    5    2
  579  349   16 1847 1257    1   16  668 7666 5531    1  761    6   13
 1026    1    1  425  478    1    4    1  327 3560   20  229    3   15
 5742    3   15 1620   15   99    5    2 3550  100   11  772 1498   12
  252  235   11  217    2  366 6454    3   58   93   11   90  102   11
 1498  177   12 

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(input_dim = vocab_size, output_dim=embedding_dim, input_length= maxlen)
])